In [5]:
import fs from 'fs';

## 2-2 コールバック

### 2.2.1　コールバックを利用した非同期APIを実行する

先に`console.log()`の方が実行され、setTimeoutが非同期で実行されていることが確認できる。

In [6]:
setTimeout(
    () => {
        console.log('1s passed')
    },
    1000
)
console.log('execute setTimeout()')

execute setTimeout()


1s passed


同様にして、`fs.readdir()`も非同期で実行されることがわかる。

ソースを見つけられなかったが、非同期処理では以下が満たされるように関数が設計されているらしい。
- コールバックが最後に引数
- コールバックの最初の引数がerr, それ以降が処理の結果

In [ ]:
fs.readdir(
    '.',
    (err, files) => {
        if (err) {
            console.log(err)
        } else {
            console.log(files)
        }
    }
)
console.log('execute fs.readdir()')

execute fs.readdir()


[ 'ch02.ipynb' ]


In [8]:
fs.readdir(
    'not-exist-directory',
    (err, files) => {
        if (err) {
            console.log(err)
        } else {
            console.log(files)
        }
    }
)
console.log('execute fs.readdir()')

execute fs.readdir()
[Error: ENOENT: no such file or directory, scandir 'not-exist-directory'] {
  errno: -2,
  code: 'ENOENT',
  syscall: 'scandir',
  path: 'not-exist-directory'
}
[Error: ENOENT: no such file or directory, scandir 'not-exist-directory'] {
  errno: -2,
  code: 'ENOENT',
  syscall: 'scandir',
  path: 'not-exist-directory'
}


### 2.2.2 エラーハンドリング

非同期関数に渡されるcallbackは、エラーがない時にnullを、あるときはエラーを受け取るようにする。

In [9]:
const parseJsonAsync = (json: string, callback: (err: Error | null, result: any) => void) => {
    setTimeout(
        () => {
            try {
                callback(null, JSON.parse(json))
            } catch (err) {
                callback(err, null)
            }
        },
        1000
    )
}

In [10]:
parseJsonAsync('invalid json', (err, result) => {
    if (err) {
        console.log(err)
    } else {
        console.log(result)
    }
})

SyntaxError: Unexpected token i in JSON at position 0
    at JSON.parse (<anonymous>)
    at Timeout._onTimeout (evalmachine.<anonymous>:6:33)
    at listOnTimeout (node:internal/timers:564:17)
    at process.processTimers (node:internal/timers:507:7)


### 2.2.3 混ぜるな危険 同期と非同期

非同期な関数は必ず非同期にし、同期的に処理が行われるようにはしない。

In [11]:
const cache = {}
const parseJsonAsyncWithCache = async (json: string, callback: (err: Error | null, result: any) => void) => {
    const cached = cache[json]
    if (cached) {
        setTimeout(() => callback(cached.err, cached.result), 0)
        return
    }
    parseJsonAsync(json, (err, result) => {
        cache[json] = { err, result }
        callback(err, result)
    })
}

## 2.3 Promise

### 2.5.2 async/await

async/awaitを使うと、コードがスッキリする。thenなどを使う方法もあるが、async/awaitを使うのがベスト？

awaitをつけておくと、返り値に依存するコードは、非同期処理が終わってから呼ばれることが以下のコードからわかる。したがって、依存関係について難しく考えなくても直感的にコーディングが可能。

In [18]:
const parseJsonAsync = (json: string) => new Promise((resolve, reject) => {
    console.log("parseJsonAsync() called")
    setTimeout(
        () => {
            try {
                console.log("parseJsonAsync() try block")
                resolve(JSON.parse(json))
                console.log("parseJsonAsync() resolve() called")
            } catch (err) {
                console.log("parseJsonAsync() catch block")
                reject(err)
                console.log("parseJsonAsync() reject() called")
            }
        },
        1000
    )
})

In [22]:
try {
    console.log("try block")
    const result = await parseJsonAsync('{"foo":1}')
    console.log("await parseJsonAsync() called")
    console.log(result)
} catch (err) {
    console.log(err)
}

try block
parseJsonAsync() called
parseJsonAsync() called
parseJsonAsync() try block
parseJsonAsync() resolve() called
await parseJsonAsync() called
{ foo: 1 }


In [21]:
try {
    const result = await parseJsonAsync('invalid json')
} catch (err) {
    console.log("err block")
    console.log(err)
}

parseJsonAsync() called


parseJsonAsync() try block
parseJsonAsync() catch block
parseJsonAsync() reject() called
err block
SyntaxError: Unexpected token i in JSON at position 0
    at JSON.parse (<anonymous>)
    at Timeout._onTimeout (evalmachine.<anonymous>:8:26)
    at listOnTimeout (node:internal/timers:564:17)
    at process.processTimers (node:internal/timers:507:7)
parseJsonAsync() catch block
parseJsonAsync() reject() called
err block
SyntaxError: Unexpected token i in JSON at position 0
    at JSON.parse (<anonymous>)
    at Timeout._onTimeout (evalmachine.<anonymous>:8:26)
    at listOnTimeout (node:internal/timers:564:17)
    at process.processTimers (node:internal/timers:507:7)


### 2.5.3 async/await構文と非同期処理の並行実行

In [23]:
const hogeAsync = async () => new Promise((resolve, reject) => {
    setTimeout(() => resolve('hoge'),
        1000
    )
})

const fugaAsync = async () => new Promise((resolve, reject) => {
    setTimeout(() => resolve('fuga'),
        1000
    )
})

const piyoAsync = async () => new Promise((resolve, reject) => {
    setTimeout(() => resolve('piyo'),
        1000
    )
})

下記の場合、1秒後にhoge、2秒後にfugaが変数として渡される。

In [25]:
const hoge = await hogeAsync()
console.log(hoge)
const fuga = await fugaAsync()
console.log(fuga)

hoge
fuga


以下のように呼ぶと、1秒後に全ての関数の結果が得られ、全ての関数が並行実行されていることが確認できる。これは重要そう。

In [29]:
const all = await Promise.all([hogeAsync(), fugaAsync(), piyoAsync()])
console.log(all)

[ 'hoge', 'fuga', 'piyo' ]


### 2.5.4 トップレベルawait

In [28]:
console.log(all)

[
  [AsyncFunction: hogeAsync],
  [AsyncFunction: fugaAsync],
  [AsyncFunction: piyoAsync]
]
